In [1]:
!pip uninstall numpy -y
!pip install numpy==1.24.4


Found existing installation: numpy 1.24.4
Uninstalling numpy-1.24.4:
  Successfully uninstalled numpy-1.24.4
  Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 5.23.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
xarray-einstats 0.9.0 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is 

In [1]:
!pip install scikit-surprise
from surprise import Dataset, Reader, SVD

In [2]:

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

In [3]:
# Load datasets
anime = pd.read_csv("anime.csv")
ratings = pd.read_csv("rating.csv")
ratings = ratings[ratings['rating'] != -1]
anime.dropna(subset=["genre"], inplace=True)
anime['genre'] = anime['genre'].fillna('').astype(str)

In [4]:
# TF-IDF and cosine similarity (Content-Based)
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(anime['genre'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
anime_indices = pd.Series(anime.index, index=anime['name']).drop_duplicates()


In [5]:
# Surprise Collaborative Filtering
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(ratings[['user_id', 'anime_id', 'rating']], reader)
trainset, _ = train_test_split(data, test_size=0.2)

In [6]:
import pickle
# Train the model
svd = SVD()
svd.fit(trainset)

# Save the model to a file
with open("svd_model.pkl", "wb") as f:
    pickle.dump(svd, f)

print("✅ SVD model saved as svd_model.pkl")

✅ SVD model saved as svd_model.pkl


In [7]:
# Recommender Functions
def get_content_recommendations(title, top_n=10):
    if title not in anime_indices:
        return ["Anime not found."]
    idx = anime_indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    anime_indices_list = [i[0] for i in sim_scores]
    return anime['name'].iloc[anime_indices_list].tolist()

def get_collaborative_recommendations(user_id, top_n=10):
    if user_id not in ratings['user_id'].unique():
        return ["User ID not found."]
    user_seen = ratings[ratings['user_id'] == user_id]['anime_id']
    unseen = [aid for aid in anime['anime_id'] if aid not in user_seen.values]
    predictions = [(aid, svd.predict(user_id, aid).est) for aid in unseen]
    predictions.sort(key=lambda x: x[1], reverse=True)
    top_ids = [x[0] for x in predictions[:top_n]]
    return anime[anime['anime_id'].isin(top_ids)]['name'].tolist()

def hybrid_recommendation(user_id, liked_anime_name, top_n=10):
    if liked_anime_name not in anime_indices:
        return ["Anime not found."]
    if user_id not in ratings['user_id'].unique():
        return ["User ID not found."]
    content_recs = get_content_recommendations(liked_anime_name, top_n=20)
    content_ids = anime[anime['name'].isin(content_recs)]['anime_id']
    predictions = [(aid, svd.predict(user_id, aid).est) for aid in content_ids]
    predictions.sort(key=lambda x: x[1], reverse=True)
    top_ids = [x[0] for x in predictions[:top_n]]
    return anime[anime['anime_id'].isin(top_ids)]['name'].tolist()


In [8]:
3import difflib

print("Welcome to the Anime Recommender System")

# --- User ID input with retry ---
for attempt in range(2):
    try:
        user_input = input("Please enter your User ID (numbers only): ")
        user_id = int(user_input)
        break
    except ValueError:
        if attempt == 0:
            print("Invalid input. Please enter a valid number.")
        else:
            print("You entered an invalid User ID twice. Exiting the program.")
            exit()

# --- Anime title input with fuzzy matching ---
def find_closest_anime(user_input_name):
    all_titles = anime['name'].dropna().tolist()
    matches = difflib.get_close_matches(user_input_name, all_titles, n=1, cutoff=0.5)
    return matches[0] if matches else None

liked_input = input("Enter an anime you liked: ")
matched_anime = find_closest_anime(liked_input)

if matched_anime is None:
    print("Sorry, we couldn't find any anime matching your input. Please try again later.")
    exit()
else:
    print(f"\nClosest match found: {matched_anime}")

# --- Recommendation mode selection ---
print("\nChoose your recommendation method:")
print("1. Content-Based (based on anime genres)")
print("2. Collaborative Filtering (based on similar users)")
print("3. Hybrid (combination of both)")

mode = input("Enter the number of your choice (1 / 2 / 3): ")

print("\nHere are your recommendations:\n")

if mode == '1':
    print("Content-Based Recommendations:\n")
    for a in get_content_recommendations(matched_anime):
        print(a)

elif mode == '2':
    print("Collaborative Recommendations:\n")
    for a in get_collaborative_recommendations(user_id):
        print(a)

else:
    print("Hybrid Recommendations:\n")
    for a in hybrid_recommendation(user_id, matched_anime):
        print(a)

print("\nThank you for using the Anime Recommender System.")

print("We hope you found something great to watch next!")


Welcome to the Anime Recommender System
Please enter your User ID (numbers only): 123456
Enter an anime you liked: GintamaÂ°

Closest match found: Gintama°

Choose your recommendation method:
1. Content-Based (based on anime genres)
2. Collaborative Filtering (based on similar users)
3. Hybrid (combination of both)
Enter the number of your choice (1 / 2 / 3): 1

Here are your recommendations:

Content-Based Recommendations:

Gintama&#039;
Gintama Movie: Kanketsu-hen - Yorozuya yo Eien Nare
Gintama&#039;: Enchousen
Gintama
Gintama: Yorinuki Gintama-san on Theater 2D
Gintama Movie: Shinyaku Benizakura-hen
Gintama: Shinyaku Benizakura-hen
Gintama: Jump Festa 2014 Special
Gintama (2017)
Gintama: Nanigoto mo Saiyo ga Kanjin nano de Tasho Senobisuru Kurai ga Choudoyoi

Thank you for using the Anime Recommender System.
We hope you found something great to watch next!


In [ ]:
# Fix compatibility
!pip uninstall -y numpy
!pip install numpy==1.24.4
!pip install scikit-surprise

# Imports
import pandas as pd
import numpy as np
import pickle
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

# Load data
anime = pd.read_csv("anime.csv")
ratings = pd.read_csv("rating.csv")
ratings = ratings[ratings['rating'] != -1]
anime.dropna(subset=["genre"], inplace=True)
anime['genre'] = anime['genre'].fillna('').astype(str)

# Content-based filtering
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(anime['genre'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
anime_indices = pd.Series(anime.index, index=anime['name']).drop_duplicates()

# Collaborative filtering
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(ratings[['user_id', 'anime_id', 'rating']], reader)
trainset, _ = train_test_split(data, test_size=0.2)
svd = SVD()
svd.fit(trainset)

# Save model
with open("svd_model.pkl", "wb") as f:
    pickle.dump(svd, f)

# Recommendation functions
def get_content_recommendations(title, top_n=10):
    if title not in anime_indices:
        return ["Anime not found."]
    idx = anime_indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    anime_indices_list = [i[0] for i in sim_scores]
    return anime['name'].iloc[anime_indices_list].tolist()

def get_collaborative_recommendations(user_id, top_n=10):
    if user_id not in ratings['user_id'].unique():
        return ["User ID not found."]
    user_seen = ratings[ratings['user_id'] == user_id]['anime_id']
    unseen = [aid for aid in anime['anime_id'] if aid not in user_seen.values]
    predictions = [(aid, svd.predict(user_id, aid).est) for aid in unseen]
    predictions.sort(key=lambda x: x[1], reverse=True)
    top_ids = [x[0] for x in predictions[:top_n]]
    return anime[anime['anime_id'].isin(top_ids)]['name'].tolist()

def hybrid_recommendation(user_id, liked_anime_name, top_n=10):
    if liked_anime_name not in anime_indices:
        return ["Anime not found."]
    if user_id not in ratings['user_id'].unique():
        return ["User ID not found."]
    content_recs = get_content_recommendations(liked_anime_name, top_n=20)
    content_ids = anime[anime['name'].isin(content_recs)]['anime_id']
    predictions = [(aid, svd.predict(user_id, aid).est) for aid in content_ids]
    predictions.sort(key=lambda x: x[1], reverse=True)
    top_ids = [x[0] for x in predictions[:top_n]]
    return anime[anime['anime_id'].isin(top_ids)]['name'].tolist()

# User input
print("Welcome to the Anime Recommender System")

# Handle User ID
for attempt in range(2):
    try:
        user_input = input("Please enter your User ID (numbers only): ")
        user_id = int(user_input)
        break
    except ValueError:
        if attempt == 0:
            print("Invalid input. Please enter a number.")
        else:
            print("Invalid twice. Exiting.")
            exit()

# Fuzzy match anime name
def find_closest_anime(user_input_name):
    all_titles = anime['name'].dropna().tolist()
    matches = difflib.get_close_matches(user_input_name, all_titles, n=1, cutoff=0.5)
    return matches[0] if matches else None

liked_input = input("Enter an anime you liked (spelling doesn't have to be exact): ")
matched_anime = find_closest_anime(liked_input)

if matched_anime is None:
    print("Anime not found. Try again later.")
    exit()
else:
    print(f"\nClosest match: {matched_anime}")

# Choose mode
print("\nChoose recommendation type:")
print("1. Content-Based")
print("2. Collaborative")
print("3. Hybrid")
mode = input("Enter 1 / 2 / 3: ")

# Show recommendations
print("\nHere are your recommendations:\n")
if mode == '1':
    for a in get_content_recommendations(matched_anime): print(a)
elif mode == '2':
    for a in get_collaborative_recommendations(user_id): print(a)
else:
    for a in hybrid_recommendation(user_id, matched_anime): print(a)

print("\nThank you for using the Anime Recommender System.")
